In [77]:
import pdfplumber
import re
import json
import os
import html
import PyPDF2
import pandas as pd


from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

## METODI

In [ ]:
load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [79]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

In [80]:
def estrai_testo_pdf(file_pdf: str) -> str:
    """
    Estrae il testo da un file PDF.
    
    Args:
        file_pdf (str): Il percorso del file PDF da cui estrarre il testo.
        
    Returns:
        str: Il testo estratto dal PDF.
    """
    testo_completo = ""
    with pdfplumber.open(file_pdf) as pdf:
        for page in pdf.pages:
            testo_completo += page.extract_text()
    
    return testo_completo

testo = estrai_testo_pdf(file_path)

In [ ]:
def analizza_menu(testo: str) -> Dict[str, Dict[str, str]]:
    """
    Analizza il testo del menu per estrarre il nome del ristorante, dello chef e del pianeta associato.

    Args:
        testo (str): Il testo completo del menu estratto dal PDF.

    Returns:
        Dict[str, Dict[str, str]]: Dizionario con:
            - La chiave è il nome del ristorante
            - Il valore è un altro dizionario con 'chef' e 'pianeta'.
    """
    # Rimuovi caratteri extra o problematici che potrebbero interferire con l'analisi
    testo = testo.replace('\n', ' ').replace('\r', '').strip()

    # Prompt per il modello LLM
    prompt = f"""
    Analizza il seguente testo del menu e estrai il nome del ristorante, lo chef e il nome del pianeta.
    La struttura del testo contiene:
    - Prima il nome del ristorante
    - Poi subito dopo il nome dello chef
    - Alla fine, il nome del pianeta associato al ristorante.

    Restituisci un oggetto JSON con:
    - La chiave è il nome del ristorante
    - Il valore è un oggetto con 'chef' e 'pianeta' come chiavi e rispettivi valori.
    
    Restituisci SOLO il JSON, senza formattazione markdown o decoratori.

    Testo:
    {testo}
    """

    # Qui invoca il modello LLM che ti restituisce un oggetto JSON (il tuo llm.invoke sarebbe qui)
    response = llm.invoke(prompt)
    content = response.content.strip()

    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('json'):
        content = content[4:].strip()

    try:
        # Cerca di caricare la risposta in un dizionario
        ristoranti = json.loads(content)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON delle informazioni sul ristorante:", content)
        ristoranti = {}

    return ristoranti


def aggiungi_ristorante(tx, nome: str, chef: str, pianeta: str):
    """
    Aggiunge un ristorante e lo collega al pianeta associato nel database Neo4j.

    Args:
        tx: Transazione Neo4j
        nome (str): Nome del ristorante
        chef (str): Nome dello chef
        pianeta (str): Nome del pianeta associato al ristorante
    """
    # Aggiunge il ristorante e associa lo chef
    query_ristorante = """
        MERGE (r:Ristorante {nome: $nome})
        SET r.chef = $chef
    """
    tx.run(query_ristorante, nome=nome, chef=chef)

    # Aggiungi il pianeta se non esiste
    query_pianeta = """
        MERGE (p:Pianeta {nome: $pianeta})
    """
    tx.run(query_pianeta, pianeta=pianeta)

    # Collega il ristorante al pianeta
    query_link_pianeta = """
        MATCH (r:Ristorante {nome: $nome})
        MATCH (p:Pianeta {nome: $pianeta})
        MERGE (r)-[:PRESENTE_SU]->(p)
    """
    tx.run(query_link_pianeta, nome=nome, pianeta=pianeta)


# Esegui l'analisi del menu
ristoranti = analizza_menu(testo)

with driver.session() as session:
    for nome, info in ristoranti.items():
        chef = info["chef"]
        pianeta = info["pianeta"]
        session.execute_write(aggiungi_ristorante, nome, chef, pianeta)


In [82]:
nome_ristorante = list(ristoranti.keys())[0]
print(nome_ristorante)

Armonia Universale


### LICENZE

In [84]:
def scarica_licenze_neo4j(uri: str, user: str, password: str) -> Dict[str, str]:
    """
    Recupera tutte le licenze esistenti nel grafo Neo4j.

    Args:
        uri (str): URI del database Neo4j.
        user (str): Nome utente di Neo4j.
        password (str): Password di Neo4j.

    Returns:
        dict: Dizionario con il nome della licenza come chiave e la descrizione come valore.
    """
    licenze = {}

    with driver.session() as session:
        query = "MATCH (l:Licenza) RETURN l.nome AS nome, l.descrizione AS descrizione"
        result = session.run(query)

        for record in result:
            nome = record["nome"]
            descrizione = record["descrizione"]
            licenze[nome] = descrizione

    return licenze


In [ ]:
def estrai_licenze_dal_menu(llm: AzureChatOpenAI, testo_menu: str) -> Dict[str, str]:
    """
    Estrae le licenze e i relativi livelli dal testo del menu usando LLM.

    Args:
        llm (AzureChatOpenAI): L'istanza del modello linguistico.
        testo_menu (str): Il testo del menu da analizzare.

    Returns:
        dict: Dizionario con licenze come chiavi e i livelli come valori.
    """
    prompt = f"""
    Analizza il seguente testo e identifica tutte le licenze acquisite dal ristorante.
    Le licenze possono essere nel formato:
    - Nome della licenza seguito dal livello (es. "Gravitazionale livello III")
    - Nome della licenza seguito dal livello numerico (es. "Quantistica livello 16")

    Restituisci un oggetto JSON con:
    - La chiave come il nome completo della licenza (es. "Gravitazionale livello III")
    - Il valore come la descrizione o un altro attributo associato alla licenza (può essere vuoto o un valore rilevante per il contesto).

    Testo:
    {testo_menu}
    """

    response = llm.invoke(prompt)
    content = response.content.strip()

    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('json'):
        content = content[4:].strip()

    try:
        licenze = json.loads(content)
    except json.JSONDecodeError:
        print("Errore nel parsing JON delle licenze:", content)
        licenze = {}

    return licenze


In [86]:
def crea_o_collega_licenze_ristorante(ristorante: str, licenze: Dict[str, str]):
    """
    Crea licenze in Neo4j e collega il ristorante alle licenze.

    Args:
        ristorante (str): Il nome del ristorante.
        licenze (dict): Dizionario con licenze da collegare al ristorante.
        uri (str): URI del database Neo4j.
        user (str): Nome utente di Neo4j.
        password (str): Password di Neo4j.
    """


    with driver.session() as session:
        # Per ogni licenza, verifichiamo se esiste già nel database
        for licenza in licenze:
            # Verifica se la licenza esiste nel database
            query_check = "MATCH (l:Licenza {nome: $nome}) RETURN l"
            result = session.run(query_check, nome=licenza)

            if not result.single():  # Se la licenza non esiste, la creiamo
                query_create = """
                MERGE (l:Licenza {nome: $nome})
                """
                session.run(query_create, nome=licenza)

            # Colleghiamo il ristorante alla licenza
            query_link = """
            MATCH (r:Ristorante {nome: $ristorante})
            MATCH (l:Licenza {nome: $licenza})
            MERGE (r)-[:HA_LICENZA]->(l)
            """
            session.run(query_link, ristorante=ristorante, licenza=licenza)

        print(f"Licenze collegate al ristorante '{ristorante}' con successo.")


In [87]:
def collega_ristorante_licenze(testo_menu: str, ristorante: str, llm: AzureChatOpenAI):
    """
    Estrae le licenze dal testo del menu, crea le licenze se non esistono e collega il ristorante alle licenze.
    
    Args:
        testo_menu (str): Il testo del menu da analizzare.
        ristorante (str): Il nome del ristorante.
        llm (AzureChatOpenAI): Istanza del modello linguistico.
        uri (str): URI del database Neo4j.
        user (str): Nome utente di Neo4j.
        password (str): Password di Neo4j.
    """
    # Estrai le licenze dal testo del menu usando l'LLM
    licenze_acquisite = estrai_licenze_dal_menu(llm, testo_menu)
    
    # Crea o collega le licenze al ristorante nel grafo Neo4j
    crea_o_collega_licenze_ristorante(ristorante, licenze_acquisite)


In [88]:
collega_ristorante_licenze(testo, nome_ristorante, llm)

Licenze collegate al ristorante 'Armonia Universale' con successo.


### PIATTI DEL MENU

In [89]:
def estrai_piatti_ingredienti_tecniche(llm: AzureChatOpenAI, testo_menu: str) -> List[Dict[str, List[str]]]:
    """
    Estrae piatti, ingredienti e tecniche dal testo del menu utilizzando LLM.
    
    Args:
        llm (AzureChatOpenAI): L'istanza del modello linguistico.
        testo_menu (str): Il testo del menu da analizzare.
    
    Returns:
        list: Una lista di dizionari, ciascuno contenente:
            - "piatto": nome del piatto,
            - "ingredienti": lista di sostanze/ingredienti,
            - "tecniche": lista di tecniche.
    """
    prompt = f"""
    Analizza il seguente testo del menu e identifica i seguenti elementi:
    - I nomi dei piatti
    - Gli ingredienti per ciascun piatto
    - Le tecniche necessarie per preparare ogni piatto

    Ogni piatto deve essere identificato, seguito dalla lista degli ingredienti e delle tecniche.

    Restituisci un oggetto JSON con il formato seguente:
    [
        {{
            "piatto": "Nome del piatto",
            "ingredienti": ["ingrediente1", "ingrediente2", ...],
            "tecniche": ["tecnica1", "tecnica2", ...]
        }},
        ...
    ]

    Testo:
    {testo_menu}
    """

    response = llm.invoke(prompt)
    content = response.content.strip()

    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('json'):
        content = content[4:].strip()

    try:
        piatti_info = json.loads(content)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON dei piatti:", content)
        piatti_info = []

    return piatti_info


In [92]:
import re

def match_nome(nome: str, lista: list) -> str:
    """
    Cerca un nome nella lista di entità, restituendo la corrispondenza migliore (se esiste).
    
    Args:
        nome (str): Il nome da cercare.
        lista (list): La lista di entità esistenti (Sostanze o Tecniche).
    
    Returns:
        str: La corrispondenza migliore trovata.
    """
    nome = nome.strip().lower()  # Rende il nome più comparabile (minuscolo, senza spazi)
    
    # Cerca corrispondenze esatte nel database (senza margini di errore)
    for entita in lista:
        if nome == entita.lower():  # Confronto esatto (senza margine di errore)
            return entita
    
    return nome  # Se non trova corrispondenza, restituisce il nome originale


def scarica_sostanze_tecniche() -> Dict[str, str]:
    """
    Scarica tutte le sostanze e tecniche esistenti da Neo4j per facilitare il matching.
    
    Args:
        uri (str): URI del database Neo4j.
        user (str): Nome utente di Neo4j.
        password (str): Password di Neo4j.
    
    Returns:
        dict: Un dizionario con chiavi "Sostanze" e "Tecniche", ciascuna contenente un elenco di nomi.
    """
    sostanze = []
    tecniche = []

    with driver.session() as session:
        # Estrai tutte le sostanze
        result_sostanze = session.run("MATCH (s:Sostanza) RETURN s.nome AS nome")
        sostanze = [record["nome"] for record in result_sostanze]
        
        # Estrai tutte le tecniche
        result_tecniche = session.run("MATCH (t:Tecnica) RETURN t.nome AS nome")
        tecniche = [record["nome"] for record in result_tecniche]
        
    return {"Sostanze": sostanze, "Tecniche": tecniche}


def collega_piatti_ingredienti_tecniche_ristorante_con_creazione_automatico(testo_menu: str, ristorante: str, llm: AzureChatOpenAI):
    """
    Estrae i piatti, gli ingredienti e le tecniche dal testo del menu, fa il controllo di esistenza su Neo4j,
    crea gli ingredienti e le tecniche mancanti e collega ogni piatto al ristorante, agli ingredienti e alle tecniche.
    
    Args:
        testo_menu (str): Il testo del menu da analizzare.
        ristorante (str): Il nome del ristorante.
        llm (AzureChatOpenAI): Istanza del modello linguistico.
        uri (str): URI del database Neo4j.
        user (str): Nome utente di Neo4j.
        password (str): Password di Neo4j.
    """
    # Scarica sostanze e tecniche esistenti da Neo4j
    entita = scarica_sostanze_tecniche()
    sostanze_esistenti = entita["Sostanze"]
    tecniche_esistenti = entita["Tecniche"]
    
    # Estrai piatti, ingredienti e tecniche dal testo del menu usando LLM
    piatti_info = estrai_piatti_ingredienti_tecniche(llm, testo_menu)
    

    with driver.session() as session:
        for piatto_info in piatti_info:
            piatto = piatto_info["piatto"]
            ingredienti = piatto_info["ingredienti"]
            tecniche = piatto_info["tecniche"]

            # Collega il piatto al ristorante
            query_link_ristorante = """
            MATCH (r:Ristorante {nome: $ristorante})
            MERGE (p:Piatto {nome: $piatto})
            MERGE (r)-[:PREPARATO_DA]->(p)
            """
            session.run(query_link_ristorante, ristorante=ristorante, piatto=piatto)

            # Gestisce gli ingredienti (sostanze) che non esistono
            for ingrediente in ingredienti:
                ingrediente_match = match_nome(ingrediente, sostanze_esistenti)  # Cerca una corrispondenza
                if ingrediente_match != ingrediente:  # Se non trovata una corrispondenza, creiamo l'entità
                    print(f"Creazione nuova sostanza: {ingrediente}")  # Log di debug
                    query_create_ingrediente = """
                    MERGE (s:Sostanza {nome: $ingrediente})
                    """
                    session.run(query_create_ingrediente, ingrediente=ingrediente)
                    sostanze_esistenti.append(ingrediente)  # Aggiungi alla lista per il matching futuro
                else:
                    print(f"Ingredient trovato: {ingrediente_match}")  # Log di debug
                query_link_ingrediente = """
                MATCH (p:Piatto {nome: $piatto})
                MATCH (s:Sostanza {nome: $ingrediente})
                MERGE (p)-[:PREPARATO_CON_INGREDIENTE]->(s)
                """
                session.run(query_link_ingrediente, piatto=piatto, ingrediente=ingrediente_match)

            # Gestisce le tecniche che non esistono
            for tecnica in tecniche:
                tecnica_match = match_nome(tecnica, tecniche_esistenti)  # Cerca una corrispondenza
                if tecnica_match != tecnica:  # Se non trovata una corrispondenza, creiamo l'entità
                    print(f"Creazione nuova tecnica: {tecnica}")  # Log di debug
                    query_create_tecnica = """
                    MERGE (t:Tecnica {nome: $tecnica})
                    """
                    session.run(query_create_tecnica, tecnica=tecnica)
                    tecniche_esistenti.append(tecnica)  # Aggiungi alla lista per il matching futuro
                else:
                    print(f"Tecnica trovata: {tecnica_match}")  # Log di debug
                query_link_tecnica = """
                MATCH (p:Piatto {nome: $piatto})
                MATCH (t:Tecnica {nome: $tecnica})
                MERGE (p)-[:PREPARATO_CON_TECNICA]->(t)
                """
                session.run(query_link_tecnica, piatto=piatto, tecnica=tecnica_match)

        print(f"Tutti i piatti sono stati collegati al ristorante '{ristorante}' con successo.")


In [ ]:
collega_piatti_ingredienti_tecniche_ristorante_con_creazione_automatico(testo, nome_ristorante, llm)

: 

: 